# Data cleaning and refinery for COOP sales orders and WS Price List

Objectives:
- Clean sales standard extraction of sales orders from COOP web shop
- Merge it with WS price List
- Simple Visuals of the data (next)

# Table of contents
The main parts of this notebook are:
1. [Load data into DataFrames](#load)
1. [Data Transformation and features engineering](#eng)
1. [Output File](#out)
    
1. [Next steps](#next)

In [ ]:
import numpy as np
import pandas as pd
import re

In [29]:
# The code was removed by DSX for sharing.

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Colour Guide:,Specials,New Products,Back in Stock,NaN,NaN,New Size,NaN,NaN,NaN,NaN
1,Code,Description,Brand,QTY,Single unit price,Units per Box,Unit price for box buy,GST,RRP (includes GST if applicable),Origin,Barcode
2,"ASIAN GROCERY (see also oils, vinegars, dressi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGMISBR2.300,Organic Miso - Brown Rice 300g,Carwari,1-5,$13.50,6+,$10.25,NO,$16.90,Japan,9368056969582
4,AGMISR2.500,Organic Miso - Red 500g,Carwari,1-5,$13.50,6+,$10.25,NO,$16.90,Japan,9368056969544


In [30]:
#Assign headers 
df_data_WS_Price_raw.columns = ['SKU', 'SKU_Desc', 'Brand', 'QTY', 'Single unit price', 'Units per Box', 'Unit price for box buy', 'GST', 'RRP (includes GST if applicable)', 'Origin', 'Barcode' ]
#drop columns
df_data_WS_Price_raw_clean = df_data_WS_Price_raw.drop(['SKU_Desc', 'Brand', 'QTY', 'Units per Box', 'Origin', 'Barcode', 'GST'], axis=1)

In [31]:

body = client_0c4374cc9ea94d83be07b65352226c15.get_object(Bucket='coop473c0fe0a8804789908dac85ac7d2e00',Key='Savings Calculation - From portal.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_sales_raw = pd.read_csv(body)
df_data_sales_raw.head()



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,Stas Svamin,NaN,NaN,NaN,NaN,NaN
2,Product code,Item description,Qty,Unit price,Weight,Subtotal
3,SPHONRA2.1,Organic Raw Honey - Australian 1KG,1,16.45,1.4 kg,16.45
4,GRMILH2.5,Organic Millet Hulled 5KG FILLED SPLIT,1.0 kg,$4.86/kg,5.08 kg,4.86


In [32]:
#assign columns
df_data_sales_raw.columns = ['SKU', 'SKU_Desc', 'QTY', 'Unit Price', 'Weight', 'Subtotal']
df_data_sales_raw_name = df_data_sales_raw

In [33]:
# add custoemr name to separate column to get 3n file normalization
# the assumption is that the name always goes next to 'Rpoduct name' text

#define a condition function
def get_name(row):
    if 'Product code' in row['SKU']:
        return row['lag']
    else:
        return np.nan

#add a row nad shift it 1 element up
df_data_sales_raw_name['lag'] = df_data_sales_raw_name['SKU'].shift(1)
#Add new column and assign name if the condition of Product name is met
df_data_sales_raw_name['Cust'] = df_data_sales_raw_name.fillna(value='NA').apply(get_name ,axis=1)
#Fill all NA to name above
df_data_sales_raw_name['Cust'].fillna(method = 'ffill', inplace = True)
#check the result
df_data_sales_raw_name.head()

,SKU,SKU_Desc,QTY,Unit Price,Weight,Subtotal,lag,Cust
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stas Svamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Product code,Item description,Qty,Unit price,Weight,Subtotal,Stas Svamin,Stas Svamin
3,SPHONRA2.1,Organic Raw Honey - Australian 1KG,1,16.45,1.4 kg,16.45,Product code,Stas Svamin
4,GRMILH2.5,Organic Millet Hulled 5KG FILLED SPLIT,1.0 kg,$4.86/kg,5.08 kg,4.86,SPHONRA2.1,Stas Svamin


In [48]:
#inner join by SKU (areas intersect)
df_sales_WS = pd.merge(df_data_sales_raw, df_data_WS_Price_raw_clean, how='inner', on=['SKU'], suffixes=('_sales', '_ws'))
df_sales_WS.head()

,SKU,SKU_Desc,QTY,Unit Price,Weight,Subtotal,lag,Cust,Single unit price,Unit price for box buy,RRP (includes GST if applicable)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
##kepp only necessary columns and drop NA
df_sales_WS.dropna(inplace = True)


In [50]:
##~~~~~~~~~~~~~Convert to num
#df_sales_WS['Weight_num'] = pd.to_numeric(df_sales_WS['Weight'].replace(regex=True,to_replace=r'[^0-9.]+',value=r''), errors='coerce')
df_sales_WS['Single unit price_num'] = pd.to_numeric(df_sales_WS['Single unit price'].replace(regex=True,to_replace=r'[^0-9.]+',value=r''), errors='raise')
df_sales_WS['Unit price for box buy_num'] = pd.to_numeric(df_sales_WS['Unit price for box buy'].replace(regex=True,to_replace=r'[^0-9.]+',value=r''), errors='raise')
df_sales_WS['RRP (includes GST if applicable)_num'] = pd.to_numeric(df_sales_WS['RRP (includes GST if applicable)'].replace(regex=True,to_replace=r'[^0-9.]+',value=r''), errors='raise')
df_sales_WS['Subtotal_num'] = pd.to_numeric(df_sales_WS['Subtotal'], errors='raise')


In [51]:
#print df_sales_WS[['SKU','Weight_num', 'QTY', 'Unit Price', 'Single unit price_num', 'Subtotal_num']] .head(50)

In [52]:
#weigh column is wrong, get a new one from SKU description
##new weight
df_sales_WS['new_weight'] = df_sales_WS['SKU'].str.split('.').str[1].str.strip()

In [53]:
#calculate real qnty
def qnty_calc_row(row):
     if  ' kg' in row['QTY']:
        return float(re.sub('[^0-9.]+', '', row['QTY']))/float(row['new_weight'])
     elif ' g' in row['QTY']:
        return float(re.sub('[^0-9.]+', '',  row['QTY']))/1000/float(row['new_weight'])
     else:
        return row['QTY']
    
df_sales_WS['qty_real'] = df_sales_WS['QTY']
df_sales_WS['qty_real'] = df_sales_WS.apply(qnty_calc_row, axis=1)


In [54]:
#check unit price
df_sales_WS['unit_price_calc'] = df_sales_WS['Subtotal_num'].astype(float)/df_sales_WS['qty_real'].astype(float)

In [55]:
#calc subtotals based on WS price, PRP Price
df_sales_WS['Subtotal_WS'] = df_sales_WS['qty_real'].astype(float) * df_sales_WS['Single unit price_num'].astype(float)
df_sales_WS['Subtotal_RRP'] = df_sales_WS['qty_real'].astype(float) * df_sales_WS['RRP (includes GST if applicable)_num'].astype(float)

In [56]:
#check
df_sales_WS[['SKU', 'Cust', 'qty_real', 'unit_price_calc', 'Single unit price_num', 'RRP (includes GST if applicable)_num', 'Subtotal_num', 'Subtotal_WS', 'Subtotal_RRP']].head()

,SKU,Cust,qty_real,unit_price_calc,Single unit price_num,RRP (includes GST if applicable)_num,Subtotal_num,Subtotal_WS,Subtotal_RRP
27,SPHONRA2.1,Stas Svamin,1,16.45,16.45,19.95,16.45,16.45,19.95
28,SPHONRA2.1,Amanda Bernstein,1,16.45,16.45,19.95,16.45,16.45,19.95
29,SPHONRA2.1,Diana Stapleton,1,16.45,16.45,19.95,16.45,16.45,19.95
30,GRMILH2.5,Stas Svamin,0.2,24.30,24.30,31.60,4.86,4.86,6.32
31,GRMILH2.5,Alina Malina,0.6,24.30,24.30,31.60,14.58,14.58,18.96


In [57]:

print df_sales_WS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 27 to 340
Data columns (total 20 columns):
SKU                                     314 non-null object
SKU_Desc                                314 non-null object
QTY                                     314 non-null object
Unit Price                              314 non-null object
Weight                                  314 non-null object
Subtotal                                314 non-null object
lag                                     314 non-null object
Cust                                    314 non-null object
Single unit price                       314 non-null object
Unit price for box buy                  314 non-null object
RRP (includes GST if applicable)        314 non-null object
Single unit price_num                   314 non-null float64
Unit price for box buy_num              314 non-null float64
RRP (includes GST if applicable)_num    314 non-null float64
Subtotal_num                            314 non-nul

<a name="out"></a>
### Export output file

In [60]:
# The code was removed by DSX for sharing.

In [61]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [62]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [63]:
upload_file_cos(credentials,'COOP_Detailed_v1.csv','COOP_Detailed_v1.csv')

File Uploaded
